In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

# Function to calculate Mean Absolute Percentage Error (MAPE)
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Load the dataset
data = pd.read_csv('Final filtered copy.csv')

# Variable that was initially used to acquire a sample from cleaned dataset
sample_data = data 

# Extracting year from settlement_date
sample_data['settlement_date'] = pd.to_datetime(sample_data['settlement_date'], format='%d/%m/%Y')
sample_data['settlement_year'] = sample_data['settlement_date'].dt.year

# Define features and target variable
X = sample_data.drop(['purchase_price', 'settlement_date'], axis=1)
y = sample_data['purchase_price']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# Define numerical and categorical features
numerical_features = ['area', 'median_house_price(2021)', 'median_house_rent(per week)', 'time_to_CBD[townhall]', 'population']
categorical_features = ['council_name', 'address', 'locality', 'post_code', 'property_type', 'area_type', 'settlement_date', 'Primary_purpose', 'region']

# Update features
categorical_features.remove('settlement_date')
numerical_features.append('settlement_year')
X = sample_data.drop(['purchase_price', 'settlement_date','median_house_price(2021)','median_house_rent(per week)','area_type','address','region','Primary_purpose','council_name'], axis=1)

# To display table that will be used for training model
X


,locality,post_code,property_type,area,time_to_CBD[townhall],population,settlement_year
0,Blacktown,2148,unit,100.0,40,47500,2022
1,Penrith,2750,unit,100.0,60,13500,2022
2,Warwick Farm,2170,unit,100.0,60,6000,2023
3,Penrith,2750,unit,100.0,60,13500,2022
4,Campbelltown,2560,unit,100.0,65,13000,2023
...,...,...,...,...,...,...,...
135452,Londonderry,2753,house,43550.0,110,4000,2022
135453,Dural,2158,house,45240.0,80,7750,2023
135454,Castlereagh,2749,house,46580.0,120,1250,2022
135455,Kurrajong,2758,house,48980.0,115,3250,2022


In [15]:
# Create preprocessing pipeline
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])

# Define Random Forest model
model_RFR = RandomForestRegressor()

# Create a pipeline with preprocessing and Random Forest model
rf_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model_RFR)])

# Define hyperparameters to tune
param_grid = {
    'model__n_estimators': [100, 200, 300, 400, 500],
    'model__max_features': [1.0, 'sqrt', 'log2'],
    'model__max_depth': [None, 10, 20, 30, 40, 50],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4],
    'model__bootstrap': [True, False]
}

# Create a pipeline with preprocessing and random forest model
ridge_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model_RFR)])

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    rf_pipeline, param_distributions=param_grid, n_iter=50, cv=3, 
    scoring='neg_mean_absolute_percentage_error', n_jobs=-1, random_state=42
)

# Fit RandomizedSearchCV to the data
random_search.fit(X_train, y_train)

# Get the best model
best_model = random_search.best_estimator_

# Evaluate the best model
train_score = best_model.score(X_train, y_train)
test_score = best_model.score(X_test, y_test)

print(f"Best Model Train Score: {train_score}")
print(f"Best Model Test Score: {test_score}")

# Predict on test set
y_pred = best_model.predict(X_test)

# Calculate MAPE
mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error:", mape)


/Users/yash/mambaforge/envs/DLCNN_A3/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best Model Train Score: 0.9141345701268799
Best Model Test Score: 0.7146567236492769
Mean Absolute Percentage Error: 19.520899718108353
